# DecisionTreeRegressor - very basic

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd

import sys
sys.path.append('..')
from utils import get_train_data, create_preprocessor, submit_test, train_test_split_temporal


In [2]:
X, y = get_train_data() # X comes out prepared with added columns

# Train-test split
X_train, X_valid, y_train, y_valid = train_test_split_temporal(X, y)

X_train = X_train.drop(columns=["date"])
X_valid = X_valid.drop(columns=["date"])


In [3]:
X_train.columns

Index(['site_id', 'latitude', 'longitude', 'tend', 't', 'ww', 'w1', 'tend24',
       'etat_sol', 'rr1', 'rr3', 'is_bank_holiday', 'year', 'month', 'day',
       'hour', 'is_weekend', 'season', 'hour_sin', 'hour_cos', 'day_sin',
       'day_cos', 'time_calm', 'time_morning', 'time_peak_hours',
       'time_working_hours', 'day_0', 'day_1', 'day_2', 'day_3', 'day_4',
       'day_5', 'day_6', 'arrondissement'],
      dtype='object')

In [ ]:
X_train[['tend', 't', 'ww', 'w1', 'tend24', 'etat_sol', 'rr1', 'rr3']].head()

,tend,t,ww,w1,tend24,etat_sol,rr1,rr3
107,-60,283.95,2,0.0,0.0,0.0,0.0,0.0
153,-60,283.95,2,0.0,0.0,0.0,0.0,0.0
201,-60,283.95,2,0.0,0.0,0.0,0.0,0.0
782,-110,293.65,3,2.0,-260.0,0.0,0.0,0.0
925,10,292.15,2,2.0,-210.0,0.0,0.0,0.0


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)


In [4]:
from sklearn.tree import DecisionTreeRegressor

# Define the pipeline
pipeline = Pipeline(steps=[
    #('preprocessor', preprocessor),# This preprocessor should always be used
    ('regressor', DecisionTreeRegressor(random_state=42))
])

# Train the pipeline
pipeline.fit(X_train, y_train)

# Evaluate the pipeline
y_pred = pipeline.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))

print(f"Mean Squared Error: {np.round(rmse, 2)}")

Mean Squared Error: 0.68


In [ ]:
from utils import preparation
file_name = "../submissions/" + "RFR_with_weather" + "_submission.csv"

X_test = pd.read_parquet("../data/final_test.parquet")
X_test = preparation(X_test)
X_test = X_test.drop(columns=["date"])

y_predict = pipeline.predict(X_test)

results = pd.DataFrame(
dict(
    Id=np.arange(y_predict.shape[0]),
    log_bike_count=y_predict,
    )
)
results.to_csv(file_name, index=False)

------------